In [49]:
import torch
import itertools

In [2]:
def create_rand(n1, n2=1, device='cuda'):
    if n2==1:
        return torch.rand((n1), device=device)
    else:
        return torch.rand((n1, n2), device=device)

In [3]:
def create_int(n1, n2=1, device='cuda'):
    if n2==1:
        return torch.randint(1, 54999, (n1,), device=device)
    else:
        return torch.randint(1, 54999, (n1, n2), device=device)

In [19]:
a=torch.randint(1, 54999, (500,))
b=torch.rand((500,))
a1=torch.randint(1, 54999, (500,))
b1=torch.rand((500,))
device='cuda'
def f(a, b, a1, b1, device='cuda'):
    a=a.to(device)
    b=b.to(device)
    a1=a1.to(device)
    b1=b1.to(device)
    v=torch.zeros((55000), device=device)
    v[a] = b
    v1=torch.zeros((55000), device=device)
    v1[a1]=b1
    return torch.inner(v, v1)
f(a, b, a1, b1)
# timeit f(a, b, a1, b1)

def f1(a, b, a1, b1):
    k = {}
    for ia, ib in zip(a, b):
        k[int(ia)] = float(ib)
    v=0
    for ia, ib in zip(a1, b1):
        v += k.get(int(ia), 0)*float(ib)
    return v

In [5]:
f(a, b, a1, b1)

tensor(3.4891, device='cuda:0')

In [6]:
def f100(a, b, a1, b1, device='cuda'):
    a=a.to(device)
    b=b.to(device)
    a1=a1.to(device)
    b1=b1.to(device)
    for i in range(100):
        v=torch.zeros((55000), device=device)
        v[a] = b
        v1=torch.zeros((55000), device=device)
        v1[a1]=b1
        r = torch.dot(v, v1)
    return r

In [131]:
def f200(a, b, a1, b1, device='cuda'):
    a=a.to(device)
    b=b.to(device)
    a1=a1.to(device)
    b1=b1.to(device)
    ind_a=torch.tensor([torch.cat([torch.tensor([i]*len(a)) for i in range(100)]).tolist(), 
                    a.repeat(100).tolist()
                    ], 
                   device=device)
    vals_a=b.repeat(100)
    v=torch.sparse_coo_tensor(ind_a, vals_a, (100, 55000), device=device)
    ind_b=torch.tensor([torch.cat([torch.tensor([i]*len(a1)) for i in range(100)]).tolist(), 
                    a1.repeat(100).tolist()
                    ], 
                   device=device)
    vals_b=b1.repeat(100)
    v1=torch.sparse_coo_tensor(ind_b, vals_b, (100, 55000), device=device)
    res=torch.sparse.mm(v.t(), v1)
    return res.to_dense()
%timeit f200(a,b,a1,b1, 'cuda:0')

518 ms ± 839 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def f2(a, b, a1, b1, device='cuda'):
    v=torch.zeros((100, 55000), device=device)
    for i in range(a.size()[0]):
        v[i][a[i]] = b[i]
    v1=torch.zeros((100, 55000), device=device)
    for i in range(a1.size()[0]):
        v1[i][a[i]] = b[i]
    return torch.matmul(v.reshape(100,1,55000), v1.reshape(100,55000,1)).squeeze(1)


In [132]:
%timeit f100(a, b, a1, b1, 'cpu')

4.62 ms ± 305 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit f1(a, b, a1, b1)

1.21 ms ± 5.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%timeit f(a, b, a1, b1, 'cpu')

346 μs ± 19.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
aa=create_int(100, 500)
aa1=create_int(100, 500)
bb=create_rand(100, 500)
bb1=create_rand(100, 500)

In [67]:
%timeit f2(aa, bb, aa1, bb1)

1.65 ms ± 6.98 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
x=create_int(500, 1, 'cpu')
y=create_rand(500, 1, 'cpu')
xx=create_int(100,500, 'cpu')
yy=create_rand(100,500, 'cpu')
v=torch.zeros((55000),device='cpu')
vv=torch.zeros((100,55000),device='cpu')

In [18]:
v[xx]=yy

In [19]:
for i in range(100):
    vv[i][xx[i]] = yy[i]

In [21]:
r=torch.matmul(v.reshape(1, 55000), vv.reshape(100,55000,1))

In [22]:
r.size(
)

torch.Size([100, 1, 1])

In [14]:
torch.cuda.device_count()

2

In [12]:
a

tensor([34673, 39187, 34475, 21921, 11017,  4129, 19256, 53189, 13449, 20570,
        33299, 54305, 46768, 17246, 53006, 24287, 32599, 14679, 29950, 39174,
        30578, 40321, 13688,  9519,  2983, 15066, 32488, 20378, 22760, 41604,
        26112, 27547, 15471, 51513, 50299, 13835, 33543, 42836, 50316,  7247,
        18558, 44319, 37961, 28487, 20806,  4227, 15373, 11104, 49818,  6684,
        16595, 30453, 19980, 26191, 49659, 48368, 13086, 47612, 27443, 32116,
        27380, 42905, 32636,   301, 20607, 52530,  5959, 10194, 30666, 41140,
         3271, 54452, 45668, 21677, 13827, 41863, 43228, 29978,  7744, 25401,
         8443, 30185, 18841,  6431, 51071, 20341, 30643, 33528, 54100, 38074,
        30992, 43024, 33610, 10501,  3219, 34851, 33879, 18719, 12445, 17590,
        54197, 48831, 52383,  6909, 40620,  6572, 49113, 31829, 14677,  5255,
        46093, 29832,  7581, 10109, 47956, 48829, 28650, 29373, 23914, 20216,
        48434, 23075, 53556, 19439, 44330, 46197, 15834, 24245, 

In [16]:
f200(a,b,a1,b1, 'cuda:0')

TypeError: tensor() takes 1 positional argument but 2 were given

In [18]:
torch.zeros((100, 55000), device='cuda')

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [105]:
ind_a=torch.tensor([torch.cat([torch.tensor([i]*len(a)) for i in range(100)]).tolist(), 
                    a.repeat(100).tolist()
                    ], 
                   device=device)
# vals_b=torch.cat([b for _ in range(100)])
vals_a=b.repeat(100)
v=torch.sparse_coo_tensor(ind_a, vals_a, (100, 55000), device=device)
ind_b=torch.tensor([torch.cat([torch.tensor([i]*len(a1)) for i in range(100)]).tolist(), 
                    a1.repeat(100).tolist()
                    ], 
                   device=device)
# vals_b=torch.cat([b for _ in range(100)])
vals_b=b1.repeat(100)
v1=torch.sparse_coo_tensor(ind_b, vals_b, (100, 55000), device=device)
res=torch.sparse.mm(v.t(), v1)

In [122]:
ind_a=torch.tensor([torch.cat([torch.tensor([i]*len(a)) for i in range(100)]).tolist(), 
                a.repeat(100).tolist()
                ], 
               device=device)
vals_a=b.repeat(100)
v=torch.sparse_coo_tensor(ind_a, vals_a, (100, 55000), device=device)
ind_b=torch.tensor([torch.cat([torch.tensor([i]*len(a1)) for i in range(100)]).tolist(), 
                a1.repeat(100).tolist()
                ], 
               device=device)
vals_b=b1.repeat(100)
v1=torch.sparse_coo_tensor(ind_b, vals_b, (100, 55000), device=device)
res=torch.sparse.mm(v1, v.t())


In [123]:
res.shape

torch.Size([100, 100])

TypeError: tensor() takes 1 positional argument but 2 were given

In [77]:
v1=torch.sparse_coo_tensor(ind_b, vals_b, device=device)

RuntimeError: indices must be sparse_dim x nnz, but got: [50000]

In [82]:
len(a.repeat(100))

50000

In [56]:
len(b[0])

TypeError: len() of a 0-d tensor

In [115]:
len(a)

500

In [116]:
len(a1)

500

torch.Tensor